In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    eloss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/3, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/3, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:13.0000 R:13.0000 rate:0.0260 aloss:0.7143 eloss:0.6480 aloss2:1.3996 exploreP:0.9987
Episode:1 meanR:34.0000 R:55.0000 rate:0.1100 aloss:0.6959 eloss:0.6280 aloss2:1.4125 exploreP:0.9933
Episode:2 meanR:27.0000 R:13.0000 rate:0.0260 aloss:0.7023 eloss:0.6072 aloss2:1.4229 exploreP:0.9920
Episode:3 meanR:24.0000 R:15.0000 rate:0.0300 aloss:0.7021 eloss:0.6113 aloss2:1.4190 exploreP:0.9905
Episode:4 meanR:24.4000 R:26.0000 rate:0.0520 aloss:0.7116 eloss:0.6123 aloss2:1.4209 exploreP:0.9880
Episode:5 meanR:26.1667 R:35.0000 rate:0.0700 aloss:0.7035 eloss:0.5907 aloss2:1.4325 exploreP:0.9846
Episode:6 meanR:24.4286 R:14.0000 rate:0.0280 aloss:0.7080 eloss:0.5982 aloss2:1.4272 exploreP:0.9832
Episode:7 meanR:23.0000 R:13.0000 rate:0.0260 aloss:0.7091 eloss:0.5933 aloss2:1.4310 exploreP:0.9820
Episode:8 meanR:25.3333 R:44.0000 rate:0.0880 aloss:0.7032 eloss:0.5588 aloss2:1.4520 exploreP:0.9777
Episode:9 meanR:24.5000 R:17.0000 rate:0.0340 aloss:0.6987 eloss:0.5628 aloss2:1.4

Episode:80 meanR:27.1358 R:13.0000 rate:0.0260 aloss:0.6839 eloss:-0.1960 aloss2:2.7843 exploreP:0.8047
Episode:81 meanR:27.0976 R:24.0000 rate:0.0480 aloss:0.6798 eloss:-0.1838 aloss2:2.7572 exploreP:0.8027
Episode:82 meanR:27.0120 R:20.0000 rate:0.0400 aloss:0.6891 eloss:-0.1881 aloss2:2.7766 exploreP:0.8012
Episode:83 meanR:26.8810 R:16.0000 rate:0.0320 aloss:0.6947 eloss:-0.1867 aloss2:2.8452 exploreP:0.7999
Episode:84 meanR:26.8471 R:24.0000 rate:0.0480 aloss:0.6955 eloss:-0.1896 aloss2:2.7351 exploreP:0.7980
Episode:85 meanR:26.8140 R:24.0000 rate:0.0480 aloss:0.6791 eloss:-0.1601 aloss2:2.7565 exploreP:0.7961
Episode:86 meanR:26.8506 R:30.0000 rate:0.0600 aloss:0.6933 eloss:-0.1868 aloss2:2.8382 exploreP:0.7938
Episode:87 meanR:26.9205 R:33.0000 rate:0.0660 aloss:0.6861 eloss:-0.1774 aloss2:2.8053 exploreP:0.7912
Episode:88 meanR:27.1124 R:44.0000 rate:0.0880 aloss:0.6889 eloss:-0.1762 aloss2:2.7911 exploreP:0.7877
Episode:89 meanR:27.1222 R:28.0000 rate:0.0560 aloss:0.6900 elos

Episode:159 meanR:36.3300 R:48.0000 rate:0.0960 aloss:0.6833 eloss:-1.1588 aloss2:5.6382 exploreP:0.6029
Episode:160 meanR:36.1800 R:33.0000 rate:0.0660 aloss:0.6799 eloss:-1.2273 aloss2:5.7422 exploreP:0.6009
Episode:161 meanR:35.9500 R:27.0000 rate:0.0540 aloss:0.6832 eloss:-1.2498 aloss2:5.8091 exploreP:0.5993
Episode:162 meanR:35.8900 R:17.0000 rate:0.0340 aloss:0.6922 eloss:-1.2750 aloss2:5.8201 exploreP:0.5983
Episode:163 meanR:35.8200 R:18.0000 rate:0.0360 aloss:0.6975 eloss:-1.2952 aloss2:5.9208 exploreP:0.5973
Episode:164 meanR:36.0300 R:34.0000 rate:0.0680 aloss:0.6893 eloss:-1.3190 aloss2:5.9526 exploreP:0.5953
Episode:165 meanR:36.2500 R:51.0000 rate:0.1020 aloss:0.6860 eloss:-1.3058 aloss2:6.0178 exploreP:0.5923
Episode:166 meanR:36.1200 R:28.0000 rate:0.0560 aloss:0.6887 eloss:-1.3781 aloss2:6.0950 exploreP:0.5907
Episode:167 meanR:36.1600 R:32.0000 rate:0.0640 aloss:0.6796 eloss:-1.4246 aloss2:6.1328 exploreP:0.5888
Episode:168 meanR:36.1400 R:50.0000 rate:0.1000 aloss:0

Episode:237 meanR:61.0300 R:39.0000 rate:0.0780 aloss:0.7678 eloss:-7.0566 aloss2:20.2395 exploreP:0.3629
Episode:238 meanR:61.0200 R:40.0000 rate:0.0800 aloss:0.7788 eloss:-6.8915 aloss2:20.2139 exploreP:0.3615
Episode:239 meanR:61.4200 R:50.0000 rate:0.1000 aloss:0.7691 eloss:-7.2903 aloss2:20.8556 exploreP:0.3598
Episode:240 meanR:61.1600 R:30.0000 rate:0.0600 aloss:0.7408 eloss:-7.3084 aloss2:20.6622 exploreP:0.3587
Episode:241 meanR:60.8400 R:27.0000 rate:0.0540 aloss:0.7817 eloss:-7.0508 aloss2:20.4010 exploreP:0.3578
Episode:242 meanR:60.2000 R:35.0000 rate:0.0700 aloss:0.7381 eloss:-7.7397 aloss2:20.6966 exploreP:0.3566
Episode:243 meanR:60.0600 R:27.0000 rate:0.0540 aloss:0.7689 eloss:-7.7393 aloss2:21.4354 exploreP:0.3556
Episode:244 meanR:60.1900 R:46.0000 rate:0.0920 aloss:0.7773 eloss:-7.7550 aloss2:21.2680 exploreP:0.3541
Episode:245 meanR:60.2000 R:27.0000 rate:0.0540 aloss:0.8324 eloss:-7.6560 aloss2:20.6018 exploreP:0.3531
Episode:246 meanR:59.8900 R:22.0000 rate:0.044

Episode:315 meanR:37.4500 R:15.0000 rate:0.0300 aloss:0.7847 eloss:-11.4537 aloss2:28.2947 exploreP:0.3045
Episode:316 meanR:36.8900 R:14.0000 rate:0.0280 aloss:0.7240 eloss:-10.7793 aloss2:29.0543 exploreP:0.3041
Episode:317 meanR:36.4400 R:14.0000 rate:0.0280 aloss:0.7637 eloss:-10.6507 aloss2:27.7151 exploreP:0.3037
Episode:318 meanR:36.0200 R:17.0000 rate:0.0340 aloss:0.8042 eloss:-10.1186 aloss2:26.3781 exploreP:0.3032
Episode:319 meanR:33.7300 R:17.0000 rate:0.0340 aloss:0.8032 eloss:-11.7413 aloss2:28.8223 exploreP:0.3027
Episode:320 meanR:30.3200 R:22.0000 rate:0.0440 aloss:0.8367 eloss:-11.2239 aloss2:27.6524 exploreP:0.3020
Episode:321 meanR:29.4400 R:16.0000 rate:0.0320 aloss:0.8057 eloss:-10.9957 aloss2:27.5737 exploreP:0.3015
Episode:322 meanR:28.3600 R:15.0000 rate:0.0300 aloss:0.7608 eloss:-10.4748 aloss2:28.1271 exploreP:0.3011
Episode:323 meanR:27.2600 R:13.0000 rate:0.0260 aloss:0.7296 eloss:-10.7602 aloss2:28.3226 exploreP:0.3007
Episode:324 meanR:26.6900 R:16.0000 r

...

Episode:1274 meanR:362.4700 R:500.0000 rate:1.0000 aloss:0.1696 eloss:-108768.0781 aloss2:109535.3125 exploreP:0.0100
Episode:1275 meanR:359.4000 R:193.0000 rate:0.3860 aloss:0.1673 eloss:-109322.9453 aloss2:109948.9297 exploreP:0.0100
Episode:1276 meanR:356.2200 R:182.0000 rate:0.3640 aloss:0.1682 eloss:-108836.6797 aloss2:109107.3516 exploreP:0.0100
Episode:1277 meanR:356.2200 R:500.0000 rate:1.0000 aloss:0.1637 eloss:-110707.2188 aloss2:111473.0703 exploreP:0.0100
Episode:1278 meanR:356.2200 R:500.0000 rate:1.0000 aloss:0.1689 eloss:-111747.9844 aloss2:112413.9219 exploreP:0.0100
Episode:1279 meanR:357.0800 R:127.0000 rate:0.2540 aloss:0.1689 eloss:-112999.0859 aloss2:114462.5391 exploreP:0.0100
Episode:1280 meanR:354.0900 R:201.0000 rate:0.4020 aloss:0.1665 eloss:-112380.4141 aloss2:113008.7734 exploreP:0.0100
Episode:1281 meanR:357.5200 R:500.0000 rate:1.0000 aloss:0.1669 eloss:-112623.7891 aloss2:113126.8984 exploreP:0.0100
Episode:1282 meanR:354.6100 R:209.0000 rate:0.4180 aloss

Episode:1344 meanR:368.4800 R:410.0000 rate:0.8200 aloss:0.1532 eloss:-168344.1406 aloss2:169357.8125 exploreP:0.0100
Episode:1345 meanR:372.0200 R:500.0000 rate:1.0000 aloss:0.1496 eloss:-168794.1094 aloss2:170796.9688 exploreP:0.0100
Episode:1346 meanR:372.0400 R:146.0000 rate:0.2920 aloss:0.1485 eloss:-172167.7188 aloss2:173120.6719 exploreP:0.0100
Episode:1347 meanR:374.6300 R:372.0000 rate:0.7440 aloss:0.1504 eloss:-171150.8750 aloss2:173824.8750 exploreP:0.0100
Episode:1348 meanR:374.2500 R:39.0000 rate:0.0780 aloss:0.1507 eloss:-174017.1719 aloss2:177056.6406 exploreP:0.0100
Episode:1349 meanR:374.8500 R:185.0000 rate:0.3700 aloss:0.1512 eloss:-171818.5312 aloss2:176673.7344 exploreP:0.0100
Episode:1350 meanR:371.3000 R:145.0000 rate:0.2900 aloss:0.1508 eloss:-171877.2656 aloss2:174159.5312 exploreP:0.0100
Episode:1351 meanR:367.6800 R:138.0000 rate:0.2760 aloss:0.1492 eloss:-172892.9688 aloss2:174480.9062 exploreP:0.0100
Episode:1352 meanR:367.6800 R:500.0000 rate:1.0000 aloss:

Episode:1414 meanR:354.8400 R:500.0000 rate:1.0000 aloss:0.1080 eloss:-240345.3125 aloss2:245152.6406 exploreP:0.0100
Episode:1415 meanR:357.9000 R:451.0000 rate:0.9020 aloss:0.1046 eloss:-243773.7969 aloss2:249858.2344 exploreP:0.0100
Episode:1416 meanR:357.9000 R:500.0000 rate:1.0000 aloss:0.1071 eloss:-244746.1094 aloss2:248238.7500 exploreP:0.0100
Episode:1417 meanR:357.9000 R:500.0000 rate:1.0000 aloss:0.1019 eloss:-246188.2188 aloss2:251614.8906 exploreP:0.0100
Episode:1418 meanR:358.3100 R:206.0000 rate:0.4120 aloss:0.1022 eloss:-247069.5156 aloss2:251287.8125 exploreP:0.0100
Episode:1419 meanR:358.4700 R:149.0000 rate:0.2980 aloss:0.1047 eloss:-247430.7656 aloss2:254768.3750 exploreP:0.0100
Episode:1420 meanR:358.4700 R:500.0000 rate:1.0000 aloss:0.1035 eloss:-249557.0156 aloss2:254889.7344 exploreP:0.0100
Episode:1421 meanR:358.4700 R:500.0000 rate:1.0000 aloss:0.1014 eloss:-251409.3906 aloss2:257527.5469 exploreP:0.0100
Episode:1422 meanR:358.7100 R:170.0000 rate:0.3400 aloss

Episode:1484 meanR:396.2600 R:500.0000 rate:1.0000 aloss:0.0787 eloss:-357744.9062 aloss2:369961.3750 exploreP:0.0100
Episode:1485 meanR:396.2600 R:500.0000 rate:1.0000 aloss:0.0775 eloss:-359407.0312 aloss2:374988.3125 exploreP:0.0100
Episode:1486 meanR:396.1900 R:170.0000 rate:0.3400 aloss:0.0778 eloss:-361434.8438 aloss2:386595.0625 exploreP:0.0100
Episode:1487 meanR:396.1900 R:500.0000 rate:1.0000 aloss:0.0776 eloss:-362854.9375 aloss2:387085.8438 exploreP:0.0100
Episode:1488 meanR:396.4300 R:132.0000 rate:0.2640 aloss:0.0788 eloss:-361069.5938 aloss2:370893.4688 exploreP:0.0100
Episode:1489 meanR:399.4900 R:500.0000 rate:1.0000 aloss:0.0767 eloss:-363865.2812 aloss2:374639.5625 exploreP:0.0100
Episode:1490 meanR:403.4800 R:500.0000 rate:1.0000 aloss:0.0751 eloss:-367909.9688 aloss2:380166.5938 exploreP:0.0100
Episode:1491 meanR:403.8300 R:206.0000 rate:0.4120 aloss:0.0769 eloss:-368499.7188 aloss2:379686.2812 exploreP:0.0100
Episode:1492 meanR:403.8300 R:500.0000 rate:1.0000 aloss

Episode:1554 meanR:416.5700 R:500.0000 rate:1.0000 aloss:0.0667 eloss:-445731.1875 aloss2:475249.2188 exploreP:0.0100
Episode:1555 meanR:416.5700 R:500.0000 rate:1.0000 aloss:0.0654 eloss:-447589.1250 aloss2:471081.2188 exploreP:0.0100
Episode:1556 meanR:413.1400 R:157.0000 rate:0.3140 aloss:0.0671 eloss:-448055.4375 aloss2:495290.0000 exploreP:0.0100
Episode:1557 meanR:409.7900 R:165.0000 rate:0.3300 aloss:0.0658 eloss:-452291.5938 aloss2:483622.0625 exploreP:0.0100
Episode:1558 meanR:409.7900 R:500.0000 rate:1.0000 aloss:0.0668 eloss:-449036.2812 aloss2:475092.2500 exploreP:0.0100
Episode:1559 meanR:413.2400 R:500.0000 rate:1.0000 aloss:0.0658 eloss:-451840.8438 aloss2:486890.5000 exploreP:0.0100
Episode:1560 meanR:409.4700 R:123.0000 rate:0.2460 aloss:0.0686 eloss:-450874.5000 aloss2:482439.4688 exploreP:0.0100
Episode:1561 meanR:409.4700 R:500.0000 rate:1.0000 aloss:0.0656 eloss:-453885.8125 aloss2:480822.9062 exploreP:0.0100
Episode:1562 meanR:409.4700 R:500.0000 rate:1.0000 aloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
